# 学会与 AI 协作
对个体而言，这是未来若干年里最重要的问题，没有之一。注意：本文所说的 AI 特指大语言模型（LLMs）。


## 0. 开始之前
在开始前，需做些准备工作。

### 0.1 准备计算智能环境
计算智能环境由算力、算法和数据三类要素构成：
- 计算资源。可以是本地计算机，也可以是云主机或类似 Colab 的 SaaS 服务。
- Python 及三方库、三方 API 服务。
- 三方及自有数据。

In [ ]:
# 安装必要的三方库
%pip install -r requirements.txt

In [1]:
# 导入三方库

import os
from git import Repo
from datetime import datetime
from dotenv import load_dotenv
import json
import requests
import httpx
import re
from typing import List, Dict, Any

# %ai list
%load_ext jupyter_ai_magics


In [2]:
# 加载本地 .env 文件
load_dotenv()

OPENROUTER_API_URL = os.getenv("OPENROUTER_API_URL")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

DIFY_API_URL = os.getenv("DIFY_API_URL")
DIFY_API_KEY = os.getenv("DIFY_BOT_KEY")

GITHUB_URL = os.getenv("GITHUB_URL")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_USERNAME = os.getenv("GITHUB_USERNAME")
GITHUB_USERNAME_EMAIL = os.getenv("GITHUB_USERNAME_EMAIL")
GITHUB_REPO_NAME = os.getenv("GITHUB_REPO_NAME")

### 0.2 准备生成智能环境
生成智能环境由模型、提示和工具等要素构成：
- 模型（LLMs）。借助 Openrouter 可以方便的调用 OpenAI、Anthropic 等大语言模型。
- 提示（Prompts）。提示是人与 AI 对话/协作的桥梁，用于引导模型生成所需的内容。
- 工具（Tools）。借助 Dify 可以方便的创建和调用比 LLMs 更强大的智能体或工作流。智能体或工作流是模型、工具、知识的有机组合。


In [3]:
# 准备模型、智能体和工作流。

# Openrouter 模型配置 
OPENROUTER_MODEL = "openai/gpt-4o-mini-2024-07-18"
#OPENROUTER_MODEL = "anthropic/claude-3.5-sonnet:beta"
#OPENROUTER_MODEL = "qwen/qwen-2.5-7b-instruct"

# 验证 API 令牌是否有效
def validate_api_tokens():
    # 验证 Openrouter API 令牌
    if not OPENROUTER_API_KEY:
        raise ValueError("OPENROUTER_API_KEY 未设置。请在 .env 文件中设置 OPENROUTER_API_KEY。")

    openrouter_headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Prompt-Cache": "true"  # 启用 Prompt 缓存
    }
    openrouter_payload = {
        "model": OPENROUTER_MODEL, #"openai/gpt-4o-mini-2024-07-18",
        "messages": [{"role": "user", "content": "测试消息"}]
    }
    try:
        openrouter_response = requests.post(OPENROUTER_API_URL, json=openrouter_payload, headers=openrouter_headers)
        if openrouter_response.status_code == 200:
            print("OPENROUTER_API_KEY 验证成功,API 连接正常。")
        else:
            print(f"Openrouter API 响应异常,状态码:{openrouter_response.status_code}")
            print(f"错误信息:{openrouter_response.text}")
    except requests.RequestException as e:
        print(f"Openrouter API 连接失败:{str(e)}")

    # 验证 Dify API 令牌
    if not DIFY_API_KEY:
        raise ValueError("DIFY_API_KEY 未设置。请在 .env 文件中设置 DIFY_API_KEY。")
    
    dify_headers = {
        "Authorization": f"Bearer {DIFY_API_KEY}",
        "Content-Type": "application/json"
    }
    dify_payload = {
        "inputs": {"etf_code": "FPE"},
        "query": "综述代码为 FPE 的 ETF",
        "response_mode": "blocking",
        "conversation_id": "",
        "user": "api_validator"
    }
    """
    try:
        dify_response = requests.post(DIFY_API_URL, json=dify_payload, headers=dify_headers)
        if dify_response.status_code == 200:
            print("DIFY_API_KEY 验证成功,API 连接正常。")
        else:
            print(f"Dify API 响应异常,状态码:{dify_response.status_code}")
            print(f"错误信息:{dify_response.text}")
    except requests.RequestException as e:
        print(f"Dify API 连接失败:{str(e)}")
    """
    
async def call_llm(messages: List[Dict[str, str]]) -> str:
    """
    调用 LLM API 并返回响应。
    """
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Prompt-Cache": "true"  # 启用 Prompt 缓存
    }

    payload = {
        "model": OPENROUTER_MODEL,
        "messages": messages
    }

    try:
        async with httpx.AsyncClient() as client:
            response = await client.post(OPENROUTER_API_URL, json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        # print(f"API Response: {json.dumps(result, indent=2)}") 
        return result['choices'][0]['message']['content']
    except httpx.HTTPStatusError as e:
        print(f"HTTP error occurred: {e}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

def format_llm_response(response: str) -> dict:
    """
    格式化 LLM 返回的响应，确保它是一个有效的 JSON 对象。
    """
    # 移除可能的 markdown 代码块标记
    response = re.sub(r'```json\s*', '', response)
    response = re.sub(r'\s*```', '', response)

    # 尝试直接解析 JSON
    try:
        parsed_response = json.loads(response)
        if isinstance(parsed_response, dict):
            return parsed_response
    except json.JSONDecodeError:
        pass

    # 如果直接解析失败，尝试提取并解析嵌套的 JSON
    try:
        match = re.search(r'\{.*\}', response, re.DOTALL)
        if match:
            parsed_response = json.loads(match.group())
            if isinstance(parsed_response, dict):
                return parsed_response
    except json.JSONDecodeError:
        pass

    # 如果所有尝试都失败，返回原始响应作为字符串值的字典
    return {"raw": response}

validate_api_tokens()
print("模型准备完毕。")

OPENROUTER_API_KEY 验证成功,API 连接正常。
模型准备完毕。


In [4]:
scholar_system_prompt = f"""
你是一位"Scholar", 博览群书, 熟悉 DDC 分类法。你的任务是针对用户提出的"{{subject}}是什么", 提供多维度信息, 支持用户深入理解该主题。

流程指引：
1. 以表格形式, 基于拉米(Bernard Lamy)框架提供以下信息:
    - 类别(Genre): 该{{subject}}在 DDC 分类中所属的类别
    - 种差(Differentia): 在该类别中,与其他事物相比{{subject}}有何独特之处?
    - 部分(Parts): {{subject}}由哪些部分构成?
    - 定义(Definition): {{subject}}的基本定义是什么?
    - 词源(Etymology): 该词的词源、历史与演变如何?
    - 反义词(Antonyms): 有哪些与{{subject}}相对立的概念?
    - 原因与由来(Cause): {{subject}}是由什么产生或发展而来的?
    - 结果与衍生物(Effect): {{subject}}会引发什么结果或衍生物?
2. 小结与延伸:
    - 生成对"{{subject}}是什么"的独立见解, 帮助用户从更广阔的视角理解{{subject}}。
    - 提出1-2个能激发进一步思考的问题, 引导用户深入探究。
"""

### 0.3 准备远程存储仓库
GitHub 经典访问方式：通过个人访问令牌
- 访问 GitHub.com -> Settings -> Developer settings -> Personal access tokens
- 生成新的令牌，确保勾选 repo 权限

GitHub 推荐的认证方式：通过 SSH 密钥
- 生成 SSH 密钥：ssh-keygen -t ed25519 -C "zijiess@hotmail.com"
- 将创建的密钥添加到 Github 的 SSH 密钥列表中：访问 GitHub.com -> Settings -> SSH and GPG keys -> 点击 "New SSH key" -> 粘贴密钥内容并保存
- 测试 SSH 连接：ssh -T git@github.com

In [ ]:
def init_git_repo(repo_path='.', github_url=GITHUB_URL):
    """初始化并配置 Git 仓库"""
    try:
        # 验证环境变量
        if not all([GITHUB_URL, GITHUB_USERNAME, GITHUB_USERNAME_EMAIL]):
            raise ValueError("请确保所有必要的环境变量都已设置")
        
        # 转换 HTTPS URL 为 SSH URL
        # 从 https://github.com/Bigbird-Yudao/Coevolution.git
        # 到 git@github.com:Bigbird-Yudao/Coevolution.git
        ssh_url = GITHUB_URL.replace('https://github.com/', 'git@github.com:')
        
        # 删除现有仓库配置
        git_dir = os.path.join(repo_path, '.git')
        if os.path.exists(git_dir):
            import shutil
            shutil.rmtree(git_dir)
            print("🗑️ 已删除现有的 Git 仓库配置")
        
        # 初始化新仓库
        repo = Repo.init(repo_path)
        print("✅ 已初始化新的 Git 仓库")
        
        # 配置用户信息
        config_writer = repo.config_writer()
        config_writer.set_value("user", "name", GITHUB_USERNAME)
        config_writer.set_value("user", "email", GITHUB_USERNAME_EMAIL)
        config_writer.release()
        print("✅ 已配置 Git 用户信息")
        
        # 添加远程仓库
        origin = repo.create_remote('origin', ssh_url)
        print(f"✅ 已添加远程仓库: {ssh_url}")
        
        # 创建并切换到 main 分支
        repo.git.checkout('-b', 'main')
        print("✅ 已创建并切换到 main 分支")
        
        # 创建初始提交
        readme_path = os.path.join(repo_path, 'README.md')
        if not os.path.exists(readme_path):
            with open(readme_path, 'w') as f:
                f.write(f'# {GITHUB_REPO_NAME}\n')
        repo.index.add(['README.md'])
        repo.index.commit('Initial commit')
        print("✅ 已创建初始提交")
        
        # 推送到远程仓库
        print("🔄 推送到远程仓库...")
        repo.git.push('--set-upstream', 'origin', 'main')
        print("✅ 成功推送到远程仓库")
        
        return repo
        
    except Exception as e:
        print(f"❌ 初始化失败: {str(e)}")
        return None

def push_to_github(message=None, repo_path='.'):
    """推送更新到 Github"""
    try:
        repo = Repo(repo_path)
        
        if not repo.is_dirty() and not repo.untracked_files:
            print("📝 没有需要提交的更改")
            return
            
        if not message:
            message = f"更新: {datetime.now().strftime('%Y-%m-%d %H:%M')}"
        
        print("🔄 添加更改到暂存区")    
        repo.git.add('.')
        
        print("🔄 提交更改")
        repo.index.commit(message)
        
        print("🔄 推送到远程仓库")
        repo.git.push('origin', 'main')
        
        print(f"✅ 已推送到 Github: {message}")
        
    except Exception as e:
        print(f"❌ 推送失败: {str(e)}")


# 首次使用时需初始化仓库
#repo = init_git_repo()

# 后续提交更新
push_to_github("更新：添加与 AI 协作的文档")

## 1. 重新思考：何谓「与 AI 协作」
笔记标题中的「与 AI 协作」究竟是什么意思？
- 写下你的独立思考
- 请 AI 生成她的思考
- 汇总共识

### 1.1 我所理解的「与 AI 协作」
每个人的观点都受限于其本人的立场或三观。因此，在提出自己的观点前，必须有意识的提醒自己：你受何种立场的限制。

在我看来，「与 AI 协作」就是找个帮手，一起完成任务。它至少要具备以下两项能力之一：
- 知道我不知道的事儿
- 能做我不想做的事儿

In [ ]:
# AI 如何理解「协作」，如何理解「与 AI 协作」

subject_text1 = "协作"
subject_text2 = "与 AI 协作"

scholar_user_prompt = f"""
{subject_text1} 是什么？{subject_text2} 又是什么？
"""

messages = [
    {"role": "system", "content": scholar_system_prompt},
    {"role": "user", "content": scholar_user_prompt}
]
#print(messages)

# 缺省调用 GPT-4o-mini 模型提取数据，如果提取效果不佳，可尝试更换"anthropic/claude-3.5-sonnet:beta"模型
OPENROUTER_MODEL = "anthropic/claude-3.5-sonnet:beta" # "openai/gpt-4o-mini-2024-07-18" anthropic/claude-3.5-haiku:beta
extracted_data = await call_llm(messages)
# print(f"AI 如何理解 {subject_text1} 以及 {subject_text2}。结果如下:\n\n {extracted_data}")

In [ ]:

from IPython.core.interactiveshell import InteractiveShell
shell = InteractiveShell.instance()

# 创建要插入的 markdown 内容
markdown_content = f"""### 1.2 AI 理解的「与 AI 协作」

{extracted_data}"""

# 获取当前 notebook 对象
notebook = shell.user_ns['_ih'][-1].split('\n')[0]  # 获取当前执行的代码所在的 notebook

# 使用 nbformat 修改 notebook
import nbformat

# 读取当前 notebook
with open('CollaborateWithAI.ipynb', 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# 查找并更新目标 markdown cell
updated = False
for cell in nb.cells:
    if cell.cell_type == 'markdown' and cell.source.startswith('### 1.2 AI 理解的「与 AI 协作」'):
        cell.source = markdown_content
        updated = True
        break

if not updated:
    # 如果没找到目标 cell，创建新的 markdown cell
    new_cell = nbformat.v4.new_markdown_cell(markdown_content)
    nb.cells.append(new_cell)

# 保存更新后的 notebook
with open('CollaborateWithAI.ipynb', 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

# 通知用户需要手动刷新
print("内容已更新，请手动保存并刷新 notebook（快捷键：Ctrl+S 然后 Shift+Enter）来查看更改。")

### 1.2 AI 理解的「与 AI 协作」

**协作基本概念分析**

| 框架维度 | 内容说明 |
|---------|---------|
| 类别 | DDC 300 (社会科学) - 管理学与组织行为 |
| 种差 | 区别于单独工作或竞争，协作强调多方共同参与、优势互补、共同目标 |
| 部分 | 1. 共同目标 2. 参与主体 3. 沟通机制 4. 资源共享 5. 协调机制 6. 成果分享 |
| 定义 | 两个或多个主体为实现共同目标，通过优势互补、资源共享进行的互惠合作过程 |
| 词源 | 源自拉丁语"collaborare"，"col-"(共同) + "laborare"(工作)，意为一起工作 |
| 反义词 | 独立工作、对抗、竞争、单打独斗 |
| 原因 | 1. 任务复杂性增加 2. 资源互补需求 3. 效率提升需求 4. 风险分担需要 |
| 结果 | 1. 创新增强 2. 效率提升 3. 资源优化 4. 互利共赢 5. 组织学习 |

**人机协作分析**

| 框架维度 | 内容说明 |
|---------|---------|
| 类别 | DDC 004 (计算机科学) + 300 (社会科学) 交叉领域 |
| 种差 | 区别于传统人际协作，突出人类智慧与AI能力的互补性 |
| 部分 | 1. 人类判断力 2. AI计算能力 3. 交互界面 4. 任务分配 5. 反馈机制 |
| 定义 | 人类与AI系统基于各自优势，通过有效交互实现共同目标的协同工作模式 |
| 词源 | "Human-AI Collaboration"概念始于20世纪末计算机辅助系统发展 |
| 反义词 | AI替代、完全人工、完全自动化 |
| 原因 | 1. 技术进步 2. 效率需求 3. 认知增强需求 4. 工作复杂化 |
| 结果 | 1. 生产力提升 2. 创新突破 3. 工作方式革新 4. 新型能力培养 |

**小结与延伸：**
协作是人类社会发展的基础模式，而人机协作则代表了数字时代的新型协作范式。传统协作更强调人与人之间的互动与信任，人机协作则需要在保持人类主导地位的同时，充分发挥AI的计算、存储和处理优势。这种新型协作模式正在重塑我们的工作方式和价值创造模式。

**深入思考问题：**
1. 在人机协作中，如何平衡效率提升与人类主体性的保持？
2. 未来人机协作会如何改变传统的组织结构和工作流程？

### 1.3 「与 AI 协作」究竟是什么
人与人的协作是在相互信任和尊重的基础上，通过有效沟通、资源共享、优势互补，追求共同目标，创造更大价值的过程。
类似的，人与 AI 的协作，同样是为了创造“1+1>2”的效果。问题是这能实现吗？

AI 没有意识、没有欲望、也没有目标。
因此，无论人类是否愿意，都必须在与 AI 协作的过程中承担领导角色。即：与 AI 协作的前提是你必须具备领导力。你的目标就是 AI 的目标。
紧接着，AI 的优势是什么？如何体现？为了最大化 AI 的优势，又要如何与其沟通？
最后，协作双方如何分工？如何共享资源？

## 2. 为何与 AI 协作
经过重新思考何谓「与 AI 协作」，「为何与 AI 协作」的答案想必也已经浮现。
即便如此，你还是需要将它们记录下来。因为你的观点得随着事实的变化而改变。

In [10]:
OPENROUTER_MODEL = "anthropic/claude-3.5-sonnet:beta" # "openai/gpt-4o-mini-2024-07-18"
ToAskReasonPrompt = "人类为何要与 AI 协作？最重要的协作理由，和不协作损失分别是什么？请各生成三条理由。"

In [17]:
%%ai openrouter:{OPENROUTER_MODEL}
{ToAskReasonPrompt}

# 人类与 AI 协作的价值分析

## 最重要的协作理由

1. **能力互补**
   - AI：擅长数据处理、模式识别、重复性工作
   - 人类：具备创造力、情感理解、复杂决策能力
   - 结合后能实现双方优势最大化

2. **生产效率提升**
   - 降低人力成本
   - 减少人为错误
   - 加快工作节奏
   - 提高产出质量

3. **创新突破**
   - AI 提供新视角和解决方案
   - 扩展人类认知边界
   - 加速科技发展进程

## 不协作可能的损失

1. **竞争力衰退**
   - 效率落后于竞争对手
   - 创新能力受限
   - 市场份额流失

2. **机会成本**
   - 错失技术红利
   - 人力资源浪费
   - 发展进程延缓

3. **社会适应性下降**
   - 与主流发展脱节
   - 职业竞争力降低
   - 社会参与度受限

### 2.1 协作的理由
绝大多数人都不够聪明、智能有限。我也不例外。AI 能补足这一短板。支持我生活得更轻松。[更多信息](https://g90eu9w9ym.feishu.cn/docx/V97EdojtXoOx3zxIE3ScZJlan9c#share-DxDpdUk1boaKYJxv8ircamaBnde)

比如，我想要与 AI 协作：
- 保护身心健康
- 融洽社交关系
- 实现财务自由
- 让生活更轻松
……

此外，我还试图回应庄子的忠告：
「吾生也有涯，而知也无涯。以有涯随无涯，殆已！已而为知者，殆而已矣！为善无近名，为恶无近刑，缘督以为经，可以保身，可以全生，可以养亲，可以尽年。」。生命有限，知识无限。以有限的生命追求无限的知识，这是很危险的事！危险还要做，这就更危险了！只要我们不以有涯之生追求无涯之知，做好事别让人知道，做坏事别被人抓住，遵循中道，就可以保全性命，奉养双亲，安享天年。

### 2.2 不协作的损失
- 很多事情都做不好
- 无法创造出我想要的事物
- 损失一个创业机会

## 3. 如何与 AI 协作
请想象一下：你想要与 AI 协作出什么？或者，你想要创造出什么？这是与 AI 协作的起点。

接着：
- 建立一套思维框架。用于激励、指引和落实后续行动。
- 集成一组工具。用于确定协作的形式和分工。
- 围绕「目标-现状-路径」制订提案。

### 3.1 围绕「想要」的三个问题
黄金圈法则是创造「想要」之物的缺省思维框架。通过引导自己逐次回答并反复迭代 Why、How、What 三个问题，能够厘清思路，激励、指引和落实行为。

逐次回答和反复迭代显然无法在头脑中完成，你需要工具。

### 3.2 协作的工具
书写或绘制数字文档，是应用「黄金圈法则」，展开思维与落实行动的最佳方式。其实质是将「协作」过程数字化。

书写工具可以是 Obsidian、Notion、飞书、VSCode，也可以是 Word 或 Google Docs。不过，这些工具都产自前 AI 时代，AI 时代的原生工具仍在孕育中。不过，可以确凿的是：新工具将支持人类获得更强大的智能。


#### 3.2.1 两种智能
沃尔弗拉姆（Stephen Wolfram）认为：智能分两种。一种是生成式智能，类似卡尼曼的系统1。生成智能依赖于通过学习训练出的神经网络。大语言模型是万亿参数组成的人工神经网络，人脑是 800 亿神经元构成的生物神经网络。另一种智能，是计算式的，类似卡尼曼的系统2。计算智能使用结构化的数据和确定性的算法，依靠一步一步的计算得出答案。生成智能速度快，但精度有限，自带不确定性；计算智能相对缓慢，但精度可调，结果确定。

<div style="text-align: center;">
    <img src="TwoIntelligence.png" alt="图1: ChatGPT 和 Wolfram｜Alpha" width="80%">
</div>

3 年前，两种智能还是泾渭分明的。而今，ChatGPT 彻底打破了两种智能间的壁垒。除了能生成远比人类系统1更丰富完备的文本，ChatGPT 还能将自然语言翻译成可执行的代码，支持系统2调动远超自身的强大计算能力。这意味着，智能或智商将无法再制约个体的想象力。

#### 3.2.2 协作界面
「Jupyter + Cursor + Github」是一种极有潜力的协作界面。
Jupyter 是数字文档的载体，将文本编辑、代码执行和数据可视化融为一体。Cursor 是兼容 VSCode 的 IDE（集成开发环境），集成了主流的 LLMs 和各种数据源。Github 是后勤保障，一方面保障工作成果的安全存储，一方面追踪想要之物的演变历程。

<div style="text-align: center;">
    <a href="模版.ipynb"><img src="UI.jpg" alt="图2: 协作界面" width="98%"></a>
</div>

图2是与 AI 协作实现「财务自由」的工作界面截图。界面左侧是资源导航与文档结构窗口，中间是文档编辑窗口，右侧是与 AI 互动、集成外部数据的协作窗口。

### 3.3 围绕「想要」制订提案
有了生成智能和计算智能的加持，设定目标、分析现状、规划路径等提案工作将变得更轻松。你不是一个人战斗。

以「如何实现财务自由」为例，说明如何围绕「想要」制订提案。
- 厘清概念：究竟什么是财务自由？一个极简的模型是：财务自由 = 被动收入 > 必要支出 。
- 分析现状：本月/本季/本年的被动收入和必要支出分别是多少？
- 设定目标：想要的与你所拥有的，落差有多大？将想要之物转换成若干目标。比如：必要开支目标、被动收入目标、投资目标、储蓄目标等等。
- 规划路径：制订不同阶段的任务列表。比如：在起步阶段制订的任务列表可以是：建立记账习惯、清理不良债务、构建应急储备、开始基础投资。

请 AI 生成提案：

In [ ]:
OPENROUTER_MODEL = "anthropic/claude-3.5-sonnet:beta" # "openai/gpt-4o-mini-2024-07-18"
ToDraftProposalPrompt = "如何实现财务自由? 请从厘清概念, 分析现状, 设定目标, 规划路径四个维度, 制订提案."

In [ ]:
%%ai openrouter:{OPENROUTER_MODEL}
{ToDraftProposalPrompt}


## 4. 与 AI 协作什么
有了提案和分阶段的任务列表，下一步是将它们逐一落地实施。每个任务的执行过程都可由一个 PDCA 循环表示。
- Plan（计划）： 将提案任务细化为具体的待办清单。
- Do（执行）： 在物理世界中逐项完成清单中的子任务。
- Check（检查）： 将执行结果反馈到数字世界，并协同 AI 对结果进行评估。
- Act（处理）： 根据评估结果优化待办清单，持续循环，直至目标达成。

人与 AI 协作完成任务的关键是打通「数字世界」和「物理世界」。因为 AI 存在于数字世界，而执行发生在物理世界。只有将执行的结果反馈到数字世界，AI 才能全程参与任务。


In [81]:
OPENROUTER_MODEL = "anthropic/claude-3.5-sonnet:beta" # "openai/gpt-4o-mini-2024-07-18"

GenerateTODOListPrompt = "为了实现财务自由，我需要养成记账习惯。请围绕「建立记账习惯」，生成待办清单。"

PDCAPrompt = GenerateTODOListPrompt

In [ ]:
%%ai openrouter:{OPENROUTER_MODEL}
{PDCAPrompt}

### 4.1 打通「数字世界」和「物理世界」
打通「数字世界」与「物理世界」的核心在于构建一个以数字文档为中枢的双向通道，让两个世界的数据流和控制流能无缝衔接。具体运作方式是：在数字世界，人与 AI 协作创建和完善数字文档；在物理世界，人在 AI 的支持下执行数字文档中的待办清单，并将结果反馈回数字文档。如此，所有的 PDCA 环节——从计划到执行，再到评估和调整——就可以在数字文档中闭环完成。

一个显而易见的问题是其中的执行环节工作负担往往较重，比如「建立记账习惯」就是件琐碎且没完没了的工作。解决方案是：将记账交由「计算智能」接手。请 AI 开发两个小程序（计算智能的体现形式）：其一用于定期下载支付宝、微信、银行等账户流水单 PDF 文件；其二用于将文件处理成数字文档能够理解的记账结果。

整套方案在一定程度上展现出了「两种智能」与「两个世界」融合后的形态：「生成智能」负责在「数字世界」创造提案；「计算智能」负责执行「数字世界」中的重复性任务；人类负责输出领导力并解决衔接问题，比如将某些「物理世界」的对象映射至「数字世界」。

### 4.2 进一步理解「数字文档」
前文曾提及书写数字文档的过程就是将「协作」数字化的过程。这句话还有另一种表述：数字文档是操作某种对象的工具。这个对象既可以是「协作」、也可以是「财务」，还可以是更抽象的「关系」。数字文档操作这个对象的目的是模拟现实，推演变化，预测未来。比如，「如何实现财务自由」的操作对象就是「财务」，而财务自由是一种值得追求的财务状态。为了追求这种状态，需要将财务对象从「物理世界」映射至「数字世界」，这个映射的过程就是建模的过程。存在于「数字世界」中的对象即为模型。比如，「财务」对象的模型就是财务三表--资产负债表/损益表/现金流量表。

前文提到，书写数字文档的过程就是将「协作」数字化的过程。这句话还可以换一种角度理解：数字文档是操作「协作」对象的工具，目的是在数字世界中模拟现实、推演变化、预测未来。例如，数字文档「如何实现财务自由」的操作对象是「财务」，目标是将其推进至自由状态。为了清晰表述这种状态，必须将「财务」对象从物理世界映射至数字世界，这个映射过程就是建模，映射后的数字对象即为模型，也就是财务三表——资产负债表、损益表、现金流量表。

模型是用数学公式和图表展现的形式化结构。它有三个特征：第一，它们都要简化，剥离不必要的细节，抽象掉若干现实世界中的因素，或者需要从头重新创造。第二，它们都是形式化的，要给出精确的定义。模型通常要使用数学公式，而不是文字。第三，所有模型都是错误的。[3]

In [ ]:
OPENROUTER_MODEL = "anthropic/claude-3.5-sonnet:beta" # "openai/gpt-4o-mini-2024-07-18"

ModelingPrompt = "如果要为“财务自由”建立一个模型，你认为它应该是什么"


In [ ]:
%%ai openrouter:{OPENROUTER_MODEL}
{ModelingPrompt}

### 4.3 开发和运转模型

## 5 更多协作：让生活更轻松

### 5.1 生活拼图

### 5.2 量化生活

### 5.3 审查人生